In [1]:
import matplotlib.pyplot as plt

import glob 
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from PIL import Image
from torch.autograd import Variable
from model import Gauss2D 
from matplotlib.colors import LinearSegmentedColormap

from captum.attr import IntegratedGradients
from captum.attr import GradientShap
from captum.attr import Occlusion
from captum.attr import NoiseTunnel
from captum.attr import visualization as viz

In /packages/parallel_studio_xe_2020/intelpython3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /packages/parallel_studio_xe_2020/intelpython3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /packages/parallel_studio_xe_2020/intelpython3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /packages/parallel_studio_xe_2020/intelpython3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will b

In [2]:
data_dir = 'data/val'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type == 'cpu':
    model=torch.load('model/model_out_gauss.pth',map_location=torch.device('cpu'))
else: 
    model=torch.load('model/model_out_gauss.pth')
model.eval()
print(device)

cuda


In [3]:
#data_dir = 'patches'
#data_transforms = transforms.Compose([transforms.Resize(11), transforms.ToTensor()]
#image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
#                                          data_transforms[x])

In [12]:
frameSize = 11
for img in glob.glob(str(data_dir + '*.png')
    img = Image.open('patches/img_0/img_0_patch_1.png').convert('RGB')
    height = np.shape(img)[0]
    width  = np.shape(img)[1]

    pad_h = frameSize - height 
    pad_w = frameSize - width 

    transform = transforms.Compose([
     transforms.Pad([pad_h,pad_w]),
     transforms.ToTensor()
    ])

    transformed_img = transform(img)
    transformed_img = transforms.ToPILImage()(transformed_img)

    data = transform(img).float()
    data = data.unsqueeze_(0)
    data = Variable(data)
    input   = data.to(device)
    
    output = model(input)
    output = F.softmax(output, dim=1)
    prediction_score, pred_label_idx = torch.topk(output, 1)

    classes = ['one','two']
    pred_label_idx.squeeze_()
    predicted_label = classes[pred_label_idx.item()]
    print('Predicted:', predicted_label, '(', prediction_score.squeeze().item(), ')')
    #plt.imshow(transformed_img)